In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Основные функции

In [ ]:
def int_to_bits(n: int) -> np.ndarray:
    """Функция принимает целое число в диапазоне [0, 255]. 
    Возвращает массив из 8 элементов – двоичное представление числа.
    Младший бит – крайний левый (с индексом 0)"""
    bits = list(f"{n:08b}")                 # str -> list
    bits = np.array(bits, dtype=np.int64)   # list -> array
    return np.flip(bits)                    # разворачиваем массив

rule_lut = int_to_bits(42)
rule_lut

In [ ]:
def bits_to_int(arr: np.ndarray) -> int:
    """Функция принимает массив – бинарное представление числа.
    Возвращает десятичное число.
    """
    value = 0
    for i in range(len(arr)):
        p = 2**i
        value += p * arr[i]
    return int(value)

bits_to_int(rule_lut)

In [ ]:
def pass_tape(tape: np.ndarray, lut: dict, iteration: int):
    """Функция принимает:
    - матрицу tape – временно́й роллаут одномерного автомата;
    - массив lut – двоичное представление правила переходов (младший бит по индексу 0);
    - положительное целое число iteration – номер текущей итерации;
    Функция заполняет ряд itaration+1 на основе ряда iteration по правилу lut.
    """
    length = tape.shape[1]
    for i in range(length-2):
        window = tape[iteration, i:i+3]
        lut_code = bits_to_int(np.flip(window))
        tape[iteration+1, i+1] = lut[lut_code]

# Конфигурирование

In [ ]:
length = 1001               # длина клеточного автомата
rule = 57                 # правила переходов

n_iterations = length // 2

# определим, в каких позициях будут заданы единичные ячейки автомата
init = np.array([
    length // 2, 
    # 2* length // 3 - 100,
    ])

## Случайная конфигурация

In [ ]:
# random_values = np.random.randint(0, 2, size=length)
# indices = [i for i in range(len(random_values)) if random_values[i]]
# init = np.array(indices)

# Запуск симуляции

In [ ]:
tape = np.zeros((n_iterations+1, length), dtype=np.uint8)
tape[0, init] = 1

lut = int_to_bits(rule)
for i in range(n_iterations):
    pass_tape(tape, lut, i)

# Визуализация

In [ ]:
image = tape.copy()

plt.style.use("ordevoir-dark")

image[image==0] = 20
image[image==1] = 130

plt.figure(figsize=(40, 20))
plt.axis('off')
plt.imshow(image, cmap='gray', vmin=0, vmax=255)

In [ ]:
## Визуализация фрагмента

In [ ]:
left = length // 2 - 100
right = length // 2 + 100
top = 0
bottom = 100

plt.figure(figsize=(40, 20))
plt.axis('off')
plt.imshow(image[top:bottom, left:right], cmap='gray', vmin=0, vmax=255)

# Сохранение изображения

In [ ]:
import uuid
from PIL import Image

image = tape.copy()

image[image==0] = 40
image[image==1] = 100

suffix = uuid.uuid4().hex[:8]
tape_uint8 = image.astype(np.uint8)

img = Image.fromarray(tape_uint8)  # 'L' для grayscale
img.save(f"rule_{rule}_{suffix}.png")